In [1]:
import pandas as pd #引入pandas 套件
import requests #引入爬蟲用套件
#使用數據集「美國各州大學平均學費」(Average cost of undergraduate student by state USA)
USU= pd.read_csv("nces330_20.csv" ,encoding="ASCII" )

In [2]:
#最新一年四年制大學平均學費
mask = (USU['Year'] == 2021) & (USU['Length'] == '4-year')
filtered_df = USU.loc[mask]
mean_value = filtered_df['Value'].mean()
print('2021美國一學期大學學費為：',mean_value,'USD')

2021美國一學期大學學費為： 18323.97609561753 USD


In [3]:
#將這些數據即時換算成台幣呢？
url = 'https://rate.bot.com.tw/xrt/flcsv/0/day'   # 牌告匯率 CSV 網址
rate = requests.get(url)   # 爬取網址內容
rate.encoding = 'utf-8'    # 調整回應訊息編碼為 utf-8，避免編碼不同造成亂碼
rt = rate.text             # 以文字模式讀取內容
rts = rt.split('\n')
rts[1]
a = rts[1].split(',')
usd_to_twd_rate = a[12]
mean_value_twd=mean_value*float(usd_to_twd_rate)
print('2021美國一學期大學學費為：',mean_value_twd,'NTD')

2021美國一學期大學學費為： 569325.9372908367 NTD


In [4]:
#比較美國公立大學與私立大學學費差異
mask1 = (USU['Type'] == 'Private') & (USU['Length'] == '4-year')
Private_fee = USU.loc[mask1]
mean_Private_fee = Private_fee['Value'].mean()
print('美國一學期私立大學學費為：',mean_Private_fee,'USD')
mask2 = (USU['Type'] == 'Public In-State') | (USU['Type'] == 'Public Out-State') & (USU['Length'] == '4-year')
Other_fee = USU.loc[mask2]
mean_Other_fee = Other_fee['Value'].mean()
print('美國一學期公立大學學費為：',mean_Other_fee,'USD')
print('美國一學期私立大學學費為：',mean_Private_fee*float(usd_to_twd_rate),'NTD')
print('美國一學期公立大學學費為：',mean_Other_fee*float(usd_to_twd_rate),'NTD')

美國一學期私立大學學費為： 18810.480662983424 USD
美國一學期公立大學學費為： 7676.8001543209875 USD
美國一學期私立大學學費為： 584441.634198895 NTD
美國一學期公立大學學費為： 238518.18079475308 NTD


In [5]:
#2021年不同州的公立大學學費
# 按州分组，并计算每个州的学费平均值和中位数
mean_by_state = Other_fee.groupby('State')['Value'].mean()
median_by_state = Other_fee.groupby('State')['Value'].median()
# 将平均值和中位数从大到小排序
mean_sorted = mean_by_state.sort_values(ascending=False)
median_sorted = median_by_state.sort_values(ascending=False)

# 输出结果
print('2021年不同州的大学学费平均值：')
print(mean_sorted)
print('\n2021年不同州的大学学费中位数：')
print(median_sorted)

2021年不同州的大学学费平均值：
State
New Hampshire           11164.884615
Vermont                 10797.038462
New Jersey              10009.000000
Delaware                10005.285714
Pennsylvania             9717.192308
Massachusetts            9620.230769
Connecticut              9324.769231
Illinois                 9203.615385
Rhode Island             9161.038462
New York                 8935.307692
Virginia                 8915.923077
Oregon                   8475.538462
South Carolina           8438.423077
Minnesota                8293.000000
Michigan                 8283.000000
Ohio                     8282.230769
Maryland                 8121.461538
Colorado                 7987.846154
Alaska                   7818.681818
Arizona                  7786.538462
California               7779.461538
Washington               7755.576923
Kentucky                 7707.576923
Alabama                  7682.500000
Indiana                  7662.807692
Hawaii                   7637.461538
Maine         

In [6]:
#2021年不同州的私立大學學費
# 按州分组，并计算每个州的学费平均值和中位数
mean_by_state = Private_fee.groupby('State')['Value'].mean()
median_by_state = Private_fee.groupby('State')['Value'].median()
# 将平均值和中位数从大到小排序
mean_sorted = mean_by_state.sort_values(ascending=False)
median_sorted = median_by_state.sort_values(ascending=False)

# 输出结果
print('2021年不同州的大学学费平均值：')
print(mean_sorted)
print('\n2021年不同州的大学学费中位数：')
print(median_sorted)

2021年不同州的大学学费平均值：
State
Massachusetts           28788.888889
District of Columbia    27723.166667
Vermont                 27107.944444
Connecticut             26678.888889
Rhode Island            26593.500000
Maryland                26503.055556
New York                26005.888889
Pennsylvania            25787.111111
Oregon                  24428.666667
Maine                   24296.055556
New Jersey              24123.111111
Louisiana               23555.000000
Washington              23524.000000
California              22753.500000
New Hampshire           21826.388889
Illinois                21704.111111
North Carolina          21317.611111
Indiana                 21122.166667
Texas                   21029.888889
Wisconsin               20588.777778
Ohio                    20573.555556
Minnesota               20376.388889
Georgia                 19382.500000
Tennessee               17986.611111
Nevada                  17868.222222
Florida                 17841.222222
Iowa          